# Predicting Student Wait Time at LAIR for Stanford’s Introductory Computer Science Courses

## Project Group:
Sachin Allums (sachino)\
Justin Blumencranz (jmb25)\
Andrew Hong (amhong)\
Mahathi Mangipudi (mahathim)

Stanford enrolls over 2500 students each year in its two introductory computer science courses: CS106A and CS106B. These students have the opportunity to make use of LaIR, a space where they can receive one-on-one help from a section leader with their code for a given assignment. Currently, section leaders of the course are recommended to spend 15 minutes on each help request to better manage the flow of assistance. The purpose of our project is to develop a model that can predict how long students have to wait to receive help based on the assignment they are completing, the time they go to LaIR, and the number of days they go before the assignment deadline, among other features. 

## Model Selection

We have chosen to implement a linear regression model, which will take in a variety of features describing the context of a single LaIR request and output an estimated wait time for the student to recieve help.

***

# Import Packages
Here we import all of the libraries/packages we will need to train and analyze our model

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras import regularizers
from tensorflow.keras.layers import LayerNormalization
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy, math
from sklearn.model_selection import train_test_split

# Make NumPy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

# Model Control Panel
Use the cell below to tweak the model for better performance

In [2]:
# Filepath to Dataset
FILEPATH = "master_database_March6_forModeling - master_database_March6 (1).csv"

# Define split sizes
TRAIN_SIZE = 0.8
CV_SIZE = 0.1
TEST_SIZE = 0.1

# Model hyperparameters
LEARNING_RATE = 5.0e-3
ITERATIONS = 100000
LAMBDA = None
B_INIT = 0.01
W_INIT = lambda n_features: np.random.uniform(-1, 1, n_features)

# Which Columns are Features
X_LABELS = [
    "numInQueue",
    "daysLeftClean",
    "bitAndKarel",
    "imagesAndGraphics",
    "mapsAndDictionaries",
    "lambdas",
    "fileReading",
    "grids",
    "strings",
    "userInteraction",
    "queuesAndStacks",
    "recursion",
    "structs",
    "objectOrientedProgramming",
    "pointersAndMemory",
    "sorting",
    "hashTables"
]

# Which Column is the label
Y_LABELS = "waitTime"


### Don't Touch Anything Below This Line ##################################
assert TRAIN_SIZE + CV_SIZE + TEST_SIZE == 1 # Ensure splits add up to 100%
assert TRAIN_SIZE > 0
assert CV_SIZE > 0
assert TEST_SIZE > 0
assert len(X_LABELS) > 0
assert LEARNING_RATE > 0

# Pre-Training Steps

Below are the neccesary functions to load in the data from our dataset

In [3]:
def split_dataset(filepath, x_labels, y_labels, training_only=False):
    # Load dataset from filepath
    filepath = 'master_database_March6_forModeling - master_database_March6 (1).csv'
    dtype = {"waitTime": int, "daysLeftClean": float, "numInQueue": float}
    dataset = pd.read_csv(filepath, dtype=dtype)

    # Split the data
    train, test = train_test_split(dataset, test_size=1-TRAIN_SIZE)
    test, cv = train_test_split(test, test_size=TEST_SIZE/(1-TRAIN_SIZE))

    # Print Split Sizes
    print(f"Total number of examples: {len(dataset)}")
    print(f"Sizes of TRAIN, CV, TEST: [{len(train)},{len(cv)},{len(test)}]")
    
    # Define Dictionaries
    TRAIN = {
        'X' : np.array(train[x_labels]),
        'Y' : np.array(train[y_labels])
    }
    
    CV = {
        'X' : np.array(cv[x_labels]),
        'Y' : np.array(cv[y_labels])
    }
    
    TEST = {
        'X' : np.array(test[x_labels]),
        'Y' : np.array(test[y_labels])
    }
    
    # Print Split Shapes
    print(f"X TRAIN Shape: {TRAIN['X'].shape}, X Type:{type(TRAIN['X'])})")
    print(f"y TRAIN Shape: {TRAIN['Y'].shape}, y Type:{type(TRAIN['Y'])})")
    print(f"X CV Shape: {CV['X'].shape}, X CV Type:{type(CV['X'])})")
    print(f"y CV Shape: {CV['Y'].shape}, y CV Type:{type(CV['Y'])})")
    print(f"X TEST Shape: {TEST['X'].shape}, X CV Type:{type(CV['X'])})")
    print(f"y TEST Shape: {TEST['Y'].shape}, y CV Type:{type(TEST['Y'])})")
    
    if training_only: return TRAIN
    else: return TRAIN, CV, TEST


# Training Utility Functions: Linear Regression
Below are the functions used to train the model using gradient decent with MSE cost.

In [4]:
def compute_cost(X, y, w, b): 
    """
    compute cost
    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters  
      b (scalar)       : model parameter
      
    Returns:
      cost (scalar): cost
    """
    m = X.shape[0]
    f_wb = (X @ w) + b
    error = (f_wb - y)
    return (error @ error) / (2 * m)

In [5]:
def compute_cost_regularized(X, y, w, b, lambda_=0): 
    """
    compute cost
    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters  
      b (scalar)       : model parameter
      
    Returns:
      cost (scalar): cost
    """
    m = X.shape[0]
    f_wb = (X @ w) + b
    error = (f_wb - y)
    regularization = (lambda_ / m) * (w @ w)
    return (error @ error) / (2 * m) + regularization

In [6]:
def compute_gradient(X, y, w, b): 
    """
    Computes the gradient for linear regression 
    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters  
      b (scalar)       : model parameter
      
    Returns:
      dj_dw (ndarray (n,)): The gradient of the cost w.r.t. the parameters w. 
      dj_db (scalar):       The gradient of the cost w.r.t. the parameter b. 
    """
    m = X.shape[0]
    error = (X @ w) + b - y
    dj_dw = (np.transpose(X) @ error) / m
    dj_db = np.sum(error) / m
    return dj_db, dj_dw 

In [7]:
def compute_gradient_regularized(X, y, w, b, lambda_=0): 
    """
    Computes the gradient for linear regression 
    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters  
      b (scalar)       : model parameter
      
    Returns:
      dj_dw (ndarray (n,)): The gradient of the cost w.r.t. the parameters w. 
      dj_db (scalar):       The gradient of the cost w.r.t. the parameter b. 
    """
    m = X.shape[0]
    error = (X @ w) + b - y
    dj_dw = ((np.transpose(X) @ error) + (lambda_ * w)) / m
    dj_db = np.sum(error) / m
    return dj_db, dj_dw 

In [8]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters): 
    """
    Performs batch gradient descent to learn w and b. Updates w and b by taking 
    num_iters gradient steps with learning rate alpha
    
    Args:
      X (ndarray (m,n))   : Data, m examples with n features
      y (ndarray (m,))    : target values
      w_in (ndarray (n,)) : initial model parameters  
      b_in (scalar)       : initial model parameter
      cost_function       : function to compute cost
      gradient_function   : function to compute the gradient
      alpha (float)       : Learning rate
      num_iters (int)     : number of iterations to run gradient descent
      
    Returns:
      w (ndarray (n,)) : Updated values of parameters 
      b (scalar)       : Updated value of parameter 
      """
    
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w = copy.deepcopy(w_in)  #avoid modifying global w within function
    b = b_in
    
    for i in range(num_iters):

        # Calculate the gradient and update the parameters
        dj_db,dj_dw = gradient_function(X, y, w, b)   ##None

        # Update Parameters using w, b, alpha and gradient
        w = w - alpha * dj_dw               ##None
        b = b - alpha * dj_db               ##None
      
        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            J_history.append( cost_function(X, y, w, b))

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]:8.2f}   ")
        
    return w, b, J_history #return final w,b and J history for graphing

In [9]:
def predict(x, w, b): 
    """
    single predict using linear regression
    Args:
      x (ndarray): Shape (n,) example with multiple features
      w (ndarray): Shape (n,) model parameters   
      b (scalar):             model parameter 
      
    Returns:
      p (scalar):  prediction
    """
    return np.dot(x, w) + b      

In [10]:
def train_linear_regression(X_train, Y_train, return_history=True):
    m, n = X_train.shape

    # Step 1: Initialize Parameters
    w = W_INIT(n)                          # Calls lambda W_INIT to populate w with n weights
    b = B_INIT                             # Scalar quantity
    iterations = ITERATIONS
    alpha = LEARNING_RATE

    # Step 2: Show Cost Pre-Training
    initial_cost = compute_cost(X_train, Y_train, w, b)
    print(f"Initial Cost: {initial_cost}")


    # Step 3: Show Gradiant Pre-Training
    tmp_dj_db, tmp_dj_dw = compute_gradient_regularized(X_train, Y_train, w, b)
    print(f'dj_db at initial w,b: {tmp_dj_db}')
    print(f'dj_dw at initial w,b: \n {tmp_dj_dw}')

    # Step 4: Run Gradient Decent
    w_final, b_final, J_hist = gradient_descent(X_train, Y_train, w, b,
                                                    compute_cost_regularized, compute_gradient_regularized, 
                                                    alpha, iterations)
    print(f"w, b found by gradient descent:\nw= {w_final}\nb= {b_final:0.2f}")


    # Step 5: Print a few predictions
    num_shown_predictions = 5
    for i in range(num_shown_predictions):
        print(f"prediction: {np.dot(X_train[i], w_final) + b_final:0.2f}, target value: {Y_train[i]}")
    
    # Step 6: Return trained weights and bias
    if return_history: return w_final, b_final, J_hist
    else: return w_final, b_final

In [11]:
# OLD Gradient Decent

# b_init = 0.01
# w_init = np.ones(len(X_LABELS))

# print(f"Type of w: {type(w_init)}, and type of b: {type(b_init)}")

# # # Compute and display cost using our pre-chosen optimal parameters. 
# # cost = compute_cost(X_train, y_train, w_init, b_init)
# # print(f'Cost at optimal w : {cost}')

# #Compute and display gradient 
# tmp_dj_db, tmp_dj_dw = compute_gradient_regularized(X_train, y_train, w_init, b_init)
# print(f'dj_db at initial w,b: {tmp_dj_db}')
# print(f'dj_dw at initial w,b: \n {tmp_dj_dw}')

# # initialize parameters
# initial_w = np.zeros_like(w_init)
# initial_b = 0.01
# iterations = ITERATIONS
# alpha = LEARNING_RATE

# # run gradient descent 
# w_final, b_final, J_hist = gradient_descent(X_train, y_train, initial_w, initial_b,
#                                                 compute_cost_regularized, compute_gradient_regularized, 
#                                                 alpha, iterations)
# print(f"b,w found by gradient descent: {b_final:0.2f},{w_final} ")
# m,_ = X_train.shape
# for i in range(5):
#     print(f"prediction: {np.dot(X_train[i], w_final) + b_final:0.2f}, target value: {y_train[i]}")

# # plot cost versus iteration  
# fig, (ax1, ax2) = plt.subplots(1, 2, constrained_layout=True, figsize=(12, 4))
# ax1.plot(J_hist)
# ax2.plot(100 + np.arange(len(J_hist[100:])), J_hist[100:])
# ax1.set_title("Cost vs. iteration");  ax2.set_title("Cost vs. iteration (tail)")
# ax1.set_ylabel('Cost')             ;  ax2.set_ylabel('Cost') 
# ax1.set_xlabel('iteration step')   ;  ax2.set_xlabel('iteration step') 
# plt.show()

# # # Plot for just numInQueue against Wait Time
# # plt.scatter(X_train, y_train)
# # plt.xlabel('Num in Queue')
# # plt.ylabel('Wait Time')
# # plt.title("Num In Queue vs Wait Time")
# # plt.axline((0, b_final), slope=w_final, linewidth=4, color='r')
# # plt.show()


# Training Utility Functions: Neural Network

In [12]:
def train_neural_network(X_train, Y_train):
    # Step 1: Define the number of features and output dimension
    n_features = X_train.shape[0]
    output_dim = 1  

    # Define learning rate
    alpha = LEARNING_RATE

    # Create a Sequential model
    model = tf.keras.Sequential([
        # Add normalization layer
        LayerNormalization(axis=1),
        # Add first hidden layer with L2 regularization and ReLU activation
        tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        # Add second hidden layer with L2 regularization and ReLU activation
        tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        # Add output layer
        tf.keras.layers.Dense(output_dim)
    ])

    # Create Adam optimizer with custom learning rate
    optimizer = tf.keras.optimizers.Adam(learning_rate=alpha)

    # Compile the model with the custom optimizer
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])


    # Fit the model to the training data
    model.fit(X_train, Y_train, epochs=1, batch_size=32)

    # Print the model summary
    model.summary()
    
    return model

# Analysis Utility Functions
Below are functions used to analyze the performance of the model on the split dataset

In [24]:
def compute_accuracy(X, Y, w, b, threshold):
    results = zip(predict(X, w, b), Y)

    sum_errors = 0
    total_errors = 0
    correct = 0
    incorrect = 0

    for x, y in results:
        error = abs(x - y)
        sum_errors += error
        total_errors += 1
        if error < threshold:
            correct += 1
        else:
            incorrect += 1
    print(f"Accuracy on {label}: Within Threshold={correct} | Not Within Threshold={incorrect} | Percent={(correct / (correct + incorrect)):0.5f} | Average Error={(sum_errors/total_errors):0.5f}")

In [25]:
def compute_error_bins(X, y, w, b):
    m = X.shape[0]

    f_wb = (X @ w) + b
    diff = abs(f_wb - y)
    
    small = diff[diff <= 2]
    medium = diff[(diff > 2) & (diff < 10)]
    large = diff[diff >= 10]
    
    return len(small), len(medium), len(large)

In [26]:
def compute_baseline_error(threshold):
    in_queue_TRAIN = np.array(train["numInQueue"])
    Y_train = np.array(train[Y_LABELS])
    in_queue_CV = np.array(cv["numInQueue"])
    Y_cross = np.array(cv[Y_LABELS])
    in_queue_TEST = np.array(test["numInQueue"])
    Y_test = np.array(test[Y_LABELS])
    
    splits = [(in_queue_TRAIN, Y_train, "TRAIN"), (in_queue_CV, Y_cross, "CV"), (in_queue_TEST, Y_test, "TEST")]
    
    for in_queue, Y, label in splits:
        in_queue *= 15 # 15 Minutes Per Person In Queue
        results = zip(in_queue, Y)

        sum_errors = 0
        total_errors = 0
        correct = 0
        incorrect = 0
        
        for x, y in results:
            error = abs(x - y)
            sum_errors += error
            total_errors += 1
            if error < threshold:
                correct += 1
            else:
                incorrect += 1
        print(f"Accuracy on {label}: Within Threshold={correct} | Not Within Threshold={incorrect} | Percent={(correct / (correct + incorrect)):0.5f} | Average Error={(sum_errors/total_errors):0.5f}")

# Training
Here we will load the dataset and train the model by calling our utility functions

In [27]:
# Step 1: Load Dataset
TRAIN, CV, TEST = split_dataset(FILEPATH, X_LABELS, Y_LABELS)

X_train = TRAIN['X']
Y_train = TRAIN['Y']

# model = train_neural_network(X_train, Y_train)
w, b, J_hist = train_linear_regression(X_train, Y_train, return_history=True)

/var/folders/zp/dhf9b4150274w8097wk48y0w0000gn/T/ipykernel_95145/1409751676.py:5: DtypeWarning: Columns (22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(filepath, dtype=dtype)


Total number of examples: 20237
Sizes of TRAIN, CV, TEST: [16189,2025,2023]
X TRAIN Shape: (16189, 17), X Type:<class 'numpy.ndarray'>)
y TRAIN Shape: (16189,), y Type:<class 'numpy.ndarray'>)
X CV Shape: (2025, 17), X CV Type:<class 'numpy.ndarray'>)
y CV Shape: (2025,), y CV Type:<class 'numpy.ndarray'>)
X TEST Shape: (2023, 17), X CV Type:<class 'numpy.ndarray'>)
y TEST Shape: (2023,), y CV Type:<class 'numpy.ndarray'>)
Initial Cost: 172.62567396067044
dj_db at initial w,b: -10.122015954235389
dj_dw at initial w,b: 
 [-200.138   -9.566   -0.148   -2.923   -4.38    -0.394   -1.229   -2.342
   -0.7     -2.214   -2.12    -2.857   -3.97    -0.703   -1.644   -1.439
   -0.079]
Iteration    0: Cost    97.59   
Iteration 10000: Cost    83.95   
Iteration 20000: Cost    83.88   
Iteration 30000: Cost    83.87   
Iteration 40000: Cost    83.86   
Iteration 50000: Cost    83.86   
Iteration 60000: Cost    83.86   
Iteration 70000: Cost    83.85   
Iteration 80000: Cost    83.85   
Iteration 90

# Analysis

In [14]:
# Step 2: Evaluate
X_CV = CV['X']
Y_CV = CV['Y'] 

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_train, Y_train)

# Print the evaluation results
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

506/506 [==============================] - 0s 393us/step - loss: 261.3916 - mae: 10.5737
Test Loss: 261.3916015625
Test Accuracy: 10.573713302612305


In [18]:
compute_baseline_error(threshold=1)

NameError: name 'train' is not defined

In [28]:
compute_accuracy(X_train, Y_train, w, b, threshold=5)

NameError: name 'label' is not defined

In [ ]:
# Compute cost function for cross validation set
print(compute_cost_regularized(X_cross, Y_cross, w_final, b_final))
small, medium, large = compute_error_bins(X_cross, Y_cross, w_final, b_final)
print(f"small error: {small}, medium error: {medium}, and large error: {large}")